In [1]:
import pandas as pd
import numpy as np

# mESC

Amadei dataset. (mouse embryos).

In [43]:
data_dir = '/home/ggorin/count_data/amadei/'

In [44]:
names = ['NE{}_{}'.format(x,str(y)) for x in ['65','75'] for y in range(1,3)]+['NE85_{}'.format(str(x)) for x in range(1,7)]

In [48]:
import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

In [49]:
meta = pd.read_excel(data_dir+'meta.xlsx')

/tmp/ipykernel_1396580/3630299904.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  meta = pd.read_excel(data_dir+'meta.xlsx')


In [53]:
meta.set_index('barcode')

,celltype,latent_time,type,UMAP_1,UMAP_2
barcode,,,,,
AAACACGGTGGTAACGCCGTGTTT,Epiblast,0.524328,Synthetic,-0.249567,0.602579
AAACACGGTGGTAACGGACAACTT,Gut precursors,0.877421,Synthetic,13.514733,0.282939
AAACACTATGGTAACGCAGTTTGC,Epiblast,0.501554,Synthetic,-2.048857,-0.758087
AAACTGTGTGGTAACGTCCCGTAG,Epiblast,0.611674,Synthetic,1.404837,1.976268
AAAGAAAGTGGTAACGAATCGGGT,Definitive endoderm,0.439383,Synthetic,-1.234815,-5.289140
...,...,...,...,...,...
TGCACCAGCTTGCAGATGTGGCTA,Epiblast,0.039232,natural,NaN,NaN
TGCGACTACTTGCAGAGGTGACAC,Epiblast,0.094287,natural,NaN,NaN
TGGACGGACTTGCAGAATCCCACG,Definitive endoderm,0.314687,natural,NaN,NaN


In [69]:
meta_bcs = meta['barcode']

In [90]:
meta_bcs_natural = meta['barcode'][meta['type']=='natural']

In [95]:
print(f'{len(meta_bcs)} annotated barcodes.\n')
print(f'{len(meta_bcs_natural)} annotated natural barcodes.\n')

data_bcs_all = []
for data_ind, dataset in enumerate(names):
    print(f'Dataset: {dataset}')
    data_str = '/home/ggorin/count_data/amadei/{}.inex.txt'.format(dataset)
    df = pd.read_csv(data_str,sep='\t',nrows = 1 )
    data_bcs = df.columns.str[-24:]
    print(f'\t{len(data_bcs)} barcodes.')
    data_bcs_in_meta = data_bcs[data_bcs.isin(meta_bcs)]
    print(f'\t{len(data_bcs_in_meta)} are in metadata.')
    data_bcs_all.append(data_bcs_in_meta)
data_bcs_all = np.concatenate(data_bcs_all)
print(f'{len(data_bcs_all)} total data barcodes in metadata.')
print(f'{len(np.unique(data_bcs_all))} are unique.')
data_bcs_in_meta = data_bcs_all[pd.Series(data_bcs_all).isin(meta_bcs_natural)]
print(f'{len(data_bcs_in_meta)} are labeled as natural in metadata.')


5419 annotated barcodes.

2653 annotated natural barcodes.

Dataset: NE65_1
	3312 barcodes.
	157 are in metadata.
Dataset: NE65_2
	2442 barcodes.
	104 are in metadata.
Dataset: NE75_1
	6250 barcodes.
	806 are in metadata.
Dataset: NE75_2
	8455 barcodes.
	548 are in metadata.
Dataset: NE85_1
	11875 barcodes.
	116 are in metadata.
Dataset: NE85_2
	9122 barcodes.
	561 are in metadata.
Dataset: NE85_3
	12397 barcodes.
	14 are in metadata.
Dataset: NE85_4
	11662 barcodes.
	84 are in metadata.
Dataset: NE85_5
	11737 barcodes.
	241 are in metadata.
Dataset: NE85_6
	12596 barcodes.
	101 are in metadata.
2732 total data barcodes in metadata.
2732 are unique.
2597 are labeled as natural in metadata.


# MEFs

Schiebinger 2019. Reprogramming.

In [7]:
# import scanpy as sc
# import anndata as ad
import h5py 

In [10]:
f = h5py.File('/home/ggorin/count_data/schiebinger/GSM3195680_D8_Dox_C1_gene_bc_mat.h5','r')

In [18]:
f['mm10'].keys()

<KeysViewHDF5 ['barcodes', 'data', 'gene_names', 'genes', 'indices', 'indptr', 'shape']>

In [24]:
f['mm10']['data']

<HDF5 dataset "data": shape (3370222,), type "<i4">

In [32]:
f['mm10']['indices'][:10]

array([27994, 27920, 27919, 27918, 27917, 27915, 27914, 27913, 27911,
       27910])

In [33]:
# np.reshape(f['mm10']['data'],f['mm10']['shape'])